<img src="https://global.utsa.edu/tec-partnership/images/logos/logotipo-horizontal-azul-transparente.png"  width="600">

## **Avance 2: Proyecto Integrador**
## Visualización interactiva de calidad de aire en AR en aplicaciones móviles con análisis y forecasting con AI y ML
### **TC5035 - Proyecto Integrador (Gpo 10)**
### **Equipo #56**
#### Tecnológico de Monterrey
---
*   NOMBRE: Paulina Escalante Campbell
*   MATRÍCULA: A01191962


### **Objetivo**
---

Agregar Objetivos


### **Dataset**
---
Abarca 170 países y más de 300 ciudades, proporciona una vista holística de la dinámica global de la calidad del aire. Enfocado en contaminantes importantes como el Monóxido de Carbono, Ozono, Dióxido de Nitrógeno y Material Particulado (PM2.5), sirve como un recurso valioso para científicos ambientales, formuladores de políticas e investigadores. Los insights derivados de este dataset empoderan a los usuarios para analizar tendencias de calidad del aire, formular políticas efectivas y contribuir a fomentar un planeta más saludable.

Global Air Quality Dataset 🌍
Comprehensive Air Quality Measurements from Major Cities Worldwide 🌍
https://www.kaggle.com/datasets/sazidthe1/global-air-pollution-data/data

### Diccionario de variables del dataset de calidad del aire

| Columna               | Descripción                                                                                 |
|-----------------------|---------------------------------------------------------------------------------------------|
| `country_name`        | Name of the Country                                                                         |
| `city_name`           | Name of the City                                                                            |
| `aqi_value`           | Overall AQI value of the city                                                               |
| `aqi_category`        | Overall AQI category of the city                                                            |
| `co_aqi_value`        | AQI value of Carbon Monoxide of the city                                                    |
| `co_aqi_category`     | AQI category of Carbon Monoxide of the city                                                 |
| `ozone_aqi_value`     | AQI value of Ozone of the city                                                              |
| `ozone_aqi_category`  | AQI category of Ozone of the city                                                           |
| `no2_aqi_value`       | AQI value of Nitrogen Dioxide of the city                                                  |
| `no2_aqi_category`    | AQI category of Nitrogen Dioxide of the city                                               |
| `pm2.5_aqi_value`     | AQI value of Particulate Matter (≤ 2.5 micrometers) of the city                             |
| `pm2.5_aqi_category`  | AQI category of Particulate Matter (≤ 2.5 micrometers) of the city                          |



In [1]:
# Setup inicial del proyecto con GPU y google drive, conectar a runtime de T4GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

from google.colab import drive
drive.mount('/content/drive')

# Asegurarse que los datos han sido copiados a este directorio de google drive
import os
DIR = "/content/drive/MyDrive/Colab Notebooks/ProyectoIntegrador"
os.chdir(DIR)

Sat Oct  4 05:03:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Imports para análisis de datos y visualizaciones
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px

In [3]:
# Importar los datos y entender las diferentes columas y valores, estos datos ya están preprocesados con el Avance 1
main_df = pd.read_csv('global_air_pollution_data_preprocessed.csv')
main_df

,country_name,city_name,aqi_value,aqi_category,co_aqi_value,co_aqi_category,ozone_aqi_value,ozone_aqi_category,no2_aqi_value,no2_aqi_category,pm2.5_aqi_value,pm2.5_aqi_category,city_freq_encoded,city_target_encoded
0,Russian Federation,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate,0.000043,51.0
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,0.000043,41.0
2,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate,0.000043,66.0
3,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good,0.000043,34.0
4,France,Punaauia,22,Good,0,Good,22,Good,0,Good,6,Good,0.000043,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23030,India,Gursahaiganj,184,Unhealthy,3,Good,154,Unhealthy,2,Good,184,Unhealthy,0.000043,184.0
23031,France,Sceaux,50,Good,1,Good,20,Good,5,Good,50,Good,0.000043,50.0
23032,India,Mormugao,50,Good,1,Good,22,Good,1,Good,50,Good,0.000043,50.0
23033,United States of America,Westerville,71,Moderate,1,Good,44,Good,2,Good,71,Moderate,0.000043,71.0


#**Construcción**
N/A



In [4]:
# Construir

#**Normalización**
N/A



In [5]:
# Normalizar

#**Selección y Extracción**
N/A



In [6]:
# Selección

# **Conclusiones**

Este segundo avance se centró en comprender